In [1]:
import os
import datetime as dt

import numpy as np
import pandas as pd
import tensorflow as tf

from constants import NP_SEED, TF_SEED
np.random.seed(NP_SEED)
tf.set_random_seed(TF_SEED)

from nn_clf import CNN_to_RNN, Dense, RAND_CNN, RNN
import etl
import util
from visualizer import plot_metrics
from ensembler import Ensembler

N_CLASS = 3
N_LOOKBACK = 8
SPLIT_RATIO = (1, 0, 0)  # train, val, test

In [2]:
%%time
etler = etl.ETL(N_LOOKBACK, SPLIT_RATIO, downsampling=False, reload=True)


Load dataset from raw data...


/home/yongwoo/anaconda3/envs/ipykernel_py36/lib/python3.6/site-packages/IPython/core/magics/execution.py:1238: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code, glob, local_ns)
/home/yongwoo/anaconda3/envs/ipykernel_py36/lib/python3.6/site-packages/IPython/core/magics/execution.py:1238: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code, glob, local_ns)


엑셀에 기재된 날들: 571 + 25(problem)
0 : 1 : 2 = 0.45 : 0.36 : 0.19

dt problem 날짜와 다음날 데이터를 제외한 엑셀에 기재된 날들 개수: 555
위 개수의 엑셀에 기재된 날들 중
val에 0days는 test에 0days 사용, 나머지는 train에 사용

  train   x (34216, 8, 249)
  train   y (34216, 1)
  train  dt (34216,)
    val   x (0,)
    val   y (0, 1)
    val  dt (0,)
   test   x (0,)
   test   y (0, 1)
   test  dt (0,)
problem   x (600, 8, 249)
problem  dt (600,)

[train data 분포]
0 : 1 : 2 = 0.78 : 0.14 : 0.074
[val data 분포]
0 : 1 : 2 = nan : nan : nan
[test data 분포]
0 : 1 : 2 = nan : nan : nan
CPU times: user 2min 15s, sys: 9.04 s, total: 2min 24s
Wall time: 59 s


/home/yongwoo/AI-Hackerton/Posco-ai-challnege/0830_last/etl.py:54: RuntimeWarning: invalid value encountered in long_scalars
  print('0 : 1 : 2 = {:.2} : {:.2} : {:.2}'.format((d0_dict['y'] == 0).sum() / len(d0_dict['y']),
/home/yongwoo/AI-Hackerton/Posco-ai-challnege/0830_last/etl.py:55: RuntimeWarning: invalid value encountered in long_scalars
  (d0_dict['y'] == 1).sum() / len(d0_dict['y']),
/home/yongwoo/AI-Hackerton/Posco-ai-challnege/0830_last/etl.py:56: RuntimeWarning: invalid value encountered in long_scalars
  (d0_dict['y'] == 2).sum() / len(d0_dict['y'])))


In [3]:
df = etler.df

In [4]:
for idx in df.columns:
    print(idx, df['swell'].corr(df[idx]))

SURFACE_ASOS_277:기온 -0.17605342238131116
SURFACE_ASOS_277:습도 0.1675732163087832
SURFACE_ASOS_277:풍속 0.04939604393538637
SURFACE_ASOS_277:강수량 0.14464496074195313
SURFACE_ASOS_277:증기압 -0.03309140963445884
SURFACE_ASOS_277:이슬점온도 -0.04155177447179297
SURFACE_ASOS_277:현지기압 0.06614235765640424
SURFACE_ASOS_277:해면기압 0.06922120318510307
SURFACE_ASOS_277:지면온도 -0.17460981340347959
SURFACE_ASOS_277:풍향sin -0.00018034137254802768
SURFACE_ASOS_277:풍향cos 0.14955791964982865
SURFACE_ASOS_106:기온 -0.17459623755657863
SURFACE_ASOS_106:습도 0.12564452965995573
SURFACE_ASOS_106:풍속 0.03608760769031203
SURFACE_ASOS_106:강수량 0.09231748219091246
SURFACE_ASOS_106:증기압 -0.06051423456119358
SURFACE_ASOS_106:이슬점온도 -0.05750231460642059
SURFACE_ASOS_106:현지기압 0.09073629697671119
SURFACE_ASOS_106:해면기압 0.09303361716196729
SURFACE_ASOS_106:지면온도 -0.1575489811830112
SURFACE_ASOS_106:풍향sin -0.08827617796211358
SURFACE_ASOS_106:풍향cos 0.03305732460299073
SURFACE_ASOS_130:기온 -0.16568835932269282
SURFACE_ASOS_130:습도 0.137027924486